# Understanding coverage problems: a spatial optimisation classic

*Levi John Wolf* [levi.john.wolf@bristol.ac.uk](mailto:levi.john.wolf@bristol.ac.uk)

In the previous notebook, we worked with a general-purpose *mathematical* programming problem for spatial optimisation, the *p-median problem*, which seeks to maximise the benefit gained from locating some number of "depots" required to service a set of demand points. We use binary *location* decision variables $y_j$ to represent whether or not a potential depot is selected (with $j=1...m$), and another set of *allocation* decision variables $z_{ij}$ to represent whether we serve demand point $i$ is served by depot $j$ (with $i=1...n$). Each trip from depot to demand site has some distance-based cost, $d_{ij}$, and we must carry $w_i$ units to demand site $i$ in order to fully satisfy its needs. With these symbols, we can state the P-Median problem as:

<div class="alert alert-success" role="alert">
<b>The P Median Problem</b>

*Minimise* 

$\sum_i^n\sum_j^m d_{ij}z_{ij}w_{i}$

*Subject To:*

1. *Locate $p$ Facilities*:&emsp;&emsp;&emsp;&emsp;&ensp;&nbsp; $ \sum_{j}^{m} y_{j} = p$
2. *Allocate Clients Once*:&emsp;&emsp;&emsp; $ \sum_{j}^{m} z_{ij} = 1 \ \ \ \ \forall i$
3. *Allocate to Open Facilities*:&emsp; $ z_{ij} \leq y_j \ \ \ \ \forall i,j$
4. *Binary Location Decisions*:&emsp; $y_j \in {0,1} \ \ \ \ \forall j$ <br>&emsp;&emsp;&emsp;&emsp;&ensp;&emsp;&emsp;&emsp;&emsp;&ensp;&emsp;&emsp;&emsp;&emsp;   $z_{ij} \in {0,1} \ \ \ \ \forall i,j$

</div>

This model is a very useful *general* model for solving spatial location-allocation problems. However, a distinct family of models, called *coverage* models, allow us to extend the $p$-median model functionality in useful ways. 

## The unique issue of *coverage*

In a $p$-median model, every demand point *must* be served by some depot. That is, all demand points are *covered* by a supply, no matter how remote the demand point is. By *covered*, we mean that the demand is able to be satisfied by the supply point, even when the distance between the demand and the supply is very large. 

In many optimisation problems, most clearly police or fire department planning, we cannot naively assume that very distant demand points are able to be *covered* by some supply. Indeed, we often need to ensure that every demand point is within some reasonable distance or travel time from the located facility: no house should be more than a 10 minute ambulance ride away from a hospital, for example. In this case, we might ask: how many hospitals would we need in order to ensure that no house is more than a 10-minute ambulance ride away? This is a kind of *coverage* problem (sometimes called a *set cover* problem), and is incredibly common in mathematics.  

To think of this practically, imagine each you have a map of your city laid out in front of you. Imagine that each hospital is like a big disc covering the area you can get to within ten minutes from that hospital. We want to place these discs onto our map in such a way that we "cover" every demand point with at least one disc. A "good" solution will ensure that we place the fewest discs possible. This is a *location set covering problem* (LSCP), since we seek to *locate* facilities necessary to *cover* a *set* of other demand points. 

Keeping with the practical example, cities do not have unlimited budgets, and hospitals are quite expensive to build and maintain. If we can only locate ten hospitals, we might want to locate those ten hospitals in such a way that maximizes the amount of people they can serve to our specified service standard. That is, we may want to try to place a fixed number of hospital-discs in order to *maximize the population covered* by these discs. In this case, we have a slightly different kind of coverage problem: we have a *maximal coverage location problem* (MCLP).

These two coverage problems are quite common in spatial optimisation, and require slightly different formulations to solve. We will develop the simpler one, the LSCP, first. 

## Specifying a location set covering problem 

The LSCP is quite simple to specify. Using the same symbols from our P-Median problem above, let us imagine we have $y_j$ sites where we can locate a potential facility, and $d_{ij}$ reflects the travel time from facility $j$ to demand point $i$. Then, let's imagine that we have some "service standard" $S$ that determines whether $d_{ij}$ is "short enough" for facility $j$ to cover demand $i$. That is, let's define a new term, $\eta_i$, which describes the facilities *in the neighborhood of $i$* that are close enough to serve $i$. Mathematically, we spell this as $\eta_i = \{j \ |\ d_{ij} \leq S\}$, which is read like: "the set of $j$ where $d_{ij}$ is less than $S$". The symbol $\{...\}$ indicates that we're talking about a set of things within the curly braces, and the vertical line in something like $a|b$ means that we're looking for $a$-type things on the left that obey the condition $b$ given on the right. With this in mind, we can define an LSCP to minimize the number of facilities we site that cover all of our demand points:

<div class="alert alert-success" role="alert">
<b>The Location Set Covering Problem</b>

*Minimise* 

$\sum_j^m y_{j}$

*Subject To:*

1. *All demands should be covered*:&emsp;&emsp;&emsp;&emsp; $ \sum_{j \in \eta_i} y_{j} \geq 1 \ \ \ \ \ \ \ \ \forall i$
2. *Binary Location Decisions*:&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; $y_j \in {0,1} \ \ \ \ \forall j$

</div>

Here, our objective function represents the number of facilities (hospitals, in our discussion above) that we locate. You can see that this problem has fewer *explicit* constraints than the P-Median problem we discussed before, but the "allocation" part of the decision is made implicitly/automatically. By constraint 1, all of the demand sites that can be served by $j$, will be served $j$. More than one supply can be serve a demand, too. And, note that distance does not directly enter into the LSCP beyond allowing us to define $\eta_i$ *before we try to solve the model*. Thus, the LSCP is easy to think of as a pure "locational" optimisation problem, without any "allocation" component that we must choose. 

## Specifying a maximal coverage location problem

The Maximum Coverage location problem (MCLP) is more similar in specification to a $p$-median problem, in that we require a few more constraints than the LSCP. Since we can choose *not* to cover any demand, we need to use an "allocation" decision variable to keep track of whether or not a given demand point is "allocated" to a supply site. Let's refer to this as $X_{i}$, which is a binary variable indicating whether $x_i$ is considered "covered." Then, we can state an MCLP by maximizing the total demand covered by $p$ facilities $y_j$: 

<div class="alert alert-success" role="alert">
<b>The Maximal Coverage Location Problem </b>

*Maximize*:

$\sum_{i}^n w_ix_i$

*Subject To:*

1. *A covered demand should be served*: $\sum_{j \in \eta_i} Y_j \geq X_i \ \ \forall i$
2. *Locate $p$ facilities*: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; $\sum_j^m Y_j = p$
3. *Binary Location Decisions*:&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; $y_j \in {0,1} \ \ \ \ \forall j$ <br>&emsp;&emsp;&emsp;&emsp;&ensp;&emsp;&emsp;&emsp;&emsp;&ensp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;  $x_{i} \in {0,1} \ \ \ \ \forall i$

 
</div>

Here, our objective function represents the total demand that we have served. You can see that this problem still has fewer *explicit* constraints than the P-Median problem we discussed before, but the "allocation" part of the decision is again made implicitly/automatically. By constraint 1, a site can only be considered "covered" if it fits within the service standard. Further, we only have a budget to locate $p$ facilities (constraint 2). Thus, the MCLP gets us a *step closer* to a $p$-median style problem, but we still have a maximum distance past which we cannot allocate, which changes the nature of the allocation decision. Further, this is a *maximization problem*, like the knapsack problem we discussed in the first notebook. This means that we seek to find the *largest population possible* that is covered by a solution. 

# Show

For this example, we will solve for parcel depots in inner-city Bristol. We will use a few more libraries this time: 

In [1]:
import pandas
import pulp
import geopandas # standard library for geographical data in Python
import numpy # standard library for numerical and array-based computing
import spopt # library for spatial optimisation

To implement an LSCP, we will start with data on the location of postcodes in the UK: 

In [2]:
postcodes = pandas.read_csv("./uk_postcodes.csv")
postcodes.head()

,outward,easting,northing,lat,lon,demand
0,AB1,383656,760468,56.735562,-2.268770,84.409506
1,AB10,392567,804537,57.131679,-2.124423,121.898496
2,AB11,394533,805406,57.139507,-2.091960,93.939568
3,AB12,393057,800339,57.093971,-2.116218,115.097898
4,AB13,387085,802538,57.113599,-2.214879,57.817527


Further, since we have geographic data (in terms of the latitude and longitude of postcode centroids), we will use `geopandas` to make maps and work with the spatial data. 

In [3]:
postcodes = geopandas.GeoDataFrame(
    postcodes, 
    geometry=geopandas.points_from_xy(postcodes.lon, postcodes.lat, crs="epsg:4326")
)

To keep the analysis tractable, we will also focus *only* on inner-city Bristol postcodes: 

In [4]:
bristol_postcodes = postcodes[postcodes.outward.str.startswith("BS")].copy()

# is the postcode in the center of the city? 
is_inner_city = bristol_postcodes.outward.str.lstrip("BS").astype(int) < 10

# if so, keep it around
bristol_innercity = bristol_postcodes[is_inner_city].copy()

To visualise, this results in only nine postcodes in the center of the city: 

In [5]:
bristol_innercity

,outward,easting,northing,lat,lon,demand,geometry
264,BS1,358981,174319,51.466347,-2.591879,119.066167,POINT (-2.59188 51.46635)
275,BS2,359125,173381,51.457924,-2.589693,95.765799,POINT (-2.58969 51.45792)
286,BS3,357061,169771,51.425314,-2.618968,115.962676,POINT (-2.61897 51.42531)
295,BS4,361147,170721,51.434147,-2.560308,102.504517,POINT (-2.56031 51.43415)
301,BS5,361600,173916,51.462907,-2.554132,74.424220,POINT (-2.55413 51.46291)
302,BS6,362063,174622,51.469287,-2.547538,104.156384,POINT (-2.54754 51.46929)
303,BS7,363099,176769,51.488663,-2.532850,76.470352,POINT (-2.53285 51.48866)
305,BS8,357130,173447,51.458373,-2.618418,116.620348,POINT (-2.61842 51.45837)
306,BS9,356886,176678,51.487400,-2.622330,112.749978,POINT (-2.62233 51.48740)


In [6]:
bristol_innercity.explore('demand', cmap='Reds')

In this case, we will assume that we are locating facilities at postcode centroids, and all client demand exists at that centroid. This is obviously an abstraction, but many real-world companies solve their location-allocation problems at exactly this level of abstraction! 
 
To build an `LSCP` object, we first need to deal with the service radius constraint. This means that we may need to *project* our data into an equidistant projection, useful for calculating accurate distances in meters. We will use the `estimate_utm_crs()` method in `geopandas` in order to convert our data into a projection that is reasonably accurate (within 1%) for the calculation of distances. 

In [7]:
bristol_innercity_utm = bristol_innercity.to_crs(
    bristol_innercity.estimate_utm_crs()
)

Then, let us solve an LSCP with a service radius of 5 kilometers: 

In [8]:
model = spopt.locate.LSCP.from_geodataframe(
    gdf_demand=bristol_innercity_utm,
    gdf_fac=bristol_innercity_utm,
    service_radius = 4000,
    demand_col="geometry", 
    facility_col="geometry",
)

This object is a special instance of the `spopt` spatial optimisation library for Python's `locate` class, which supports generic location-allocation problem specifications: 

In [9]:
model

You can see that the underlying `pulp` model is stored in the `.problem` attribute, and it should look similar to the problem we specified above. But, it will be *rather long* because of the number of constraints: 

In [10]:
model.problem

LSCP:
MINIMIZE
1*y_0_ + 1*y_1_ + 1*y_2_ + 1*y_3_ + 1*y_4_ + 1*y_5_ + 1*y_6_ + 1*y_7_ + 1*y_8_ + 0
SUBJECT TO
_C1: y_0_ + y_1_ + y_4_ + y_5_ + y_7_ + y_8_ >= 1

_C2: y_0_ + y_1_ + y_3_ + y_4_ + y_5_ + y_7_ + y_8_ >= 1

_C3: y_2_ + y_7_ >= 1

_C4: y_1_ + y_3_ + y_4_ >= 1

_C5: y_0_ + y_1_ + y_3_ + y_4_ + y_5_ + y_6_ >= 1

_C6: y_0_ + y_1_ + y_4_ + y_5_ + y_6_ >= 1

_C7: y_4_ + y_5_ + y_6_ >= 1

_C8: y_0_ + y_1_ + y_2_ + y_7_ + y_8_ >= 1

_C9: y_0_ + y_1_ + y_7_ + y_8_ >= 1

VARIABLES
0 <= y_0_ <= 1 Integer
0 <= y_1_ <= 1 Integer
0 <= y_2_ <= 1 Integer
0 <= y_3_ <= 1 Integer
0 <= y_4_ <= 1 Integer
0 <= y_5_ <= 1 Integer
0 <= y_6_ <= 1 Integer
0 <= y_7_ <= 1 Integer
0 <= y_8_ <= 1 Integer

Remember our P-median problem, with over 90 constraints? Here we only have 18. Note that each of the constraints corresponding to (C2) in our model are of different lengths. Take, for instance, `_C7`: this means that either `y_4`, `y_5`, or `y_6` must be chosen, since these three facilities are the only ones that cover demand point 7. Likewise, we see that only $y_2$ or $y_7$ can cover demand 3, represented in `_C3`.  

To solve the model, we use the `.solve()` method (like before).

We will again use the open-source COIN-OR solver here, `pulp.COIN_CMD()`. This also allows you to provide options to the solver itself, such as `msg=False`, which turns off the verbose printing of model progress: 

In [11]:
model.solve(pulp.COIN_CMD(msg=False))

For the solved `PMedian` object, the `.cli2fac` list records the facility that each client was assigned to in the optimal solution. You can see this below:  

In [12]:
model.cli2fac

[[4, 7], [4, 7], [7], [4], [4], [4], [4], [7], [7]]

Now, we actually have a multiple assignment! Since facilities 4 and 7 have been chosen *and* facilities 4 and 7 both cover demand 1, the `.cli2fac` entry for demand `1` has two elements! This indicates "duplicate coverage," since client 1 can be covered by both facility 4 and facility 7. 

In addition, the mapping *in reverse* is stored in the `.fac2cli` list-of-lists: 

In [13]:
model.fac2cli

[[], [], [], [], [0, 1, 3, 4, 5, 6], [], [], [0, 1, 2, 7, 8], []]

Now that we have duplicate coverages, we cannot stack the allocations easily into a single vector. instead, let's put them into a `pandas.Series`:

In [14]:
allocations = pandas.Series(model.cli2fac)
allocations

0    [4, 7]
1    [4, 7]
2       [7]
3       [4]
4       [4]
5       [4]
6       [4]
7       [7]
8       [7]
dtype: object

Then, we can merge these back to the original dataframe: 

In [15]:
bristol_innercity.assign(allocation = allocations.values)

,outward,easting,northing,lat,lon,demand,geometry,allocation
264,BS1,358981,174319,51.466347,-2.591879,119.066167,POINT (-2.59188 51.46635),"[4, 7]"
275,BS2,359125,173381,51.457924,-2.589693,95.765799,POINT (-2.58969 51.45792),"[4, 7]"
286,BS3,357061,169771,51.425314,-2.618968,115.962676,POINT (-2.61897 51.42531),[7]
295,BS4,361147,170721,51.434147,-2.560308,102.504517,POINT (-2.56031 51.43415),[4]
301,BS5,361600,173916,51.462907,-2.554132,74.424220,POINT (-2.55413 51.46291),[4]
302,BS6,362063,174622,51.469287,-2.547538,104.156384,POINT (-2.54754 51.46929),[4]
303,BS7,363099,176769,51.488663,-2.532850,76.470352,POINT (-2.53285 51.48866),[4]
305,BS8,357130,173447,51.458373,-2.618418,116.620348,POINT (-2.61842 51.45837),[7]
306,BS9,356886,176678,51.487400,-2.622330,112.749978,POINT (-2.62233 51.48740),[7]


In order to visualize our solution, we may need to convert these assignments into a categorical variable, so that folium will correctly render the multiple assignments: 

In [16]:
bristol_innercity.assign(
    allocation = allocations.astype(str).values
).explore('allocation',
         marker_kwds=dict(radius=10),
          cmap='Dark2'
         )

The process of fitting an MCLP is like a mixture of the LSCP and the P-Median. That is, we require *both* a service radius and a maximum number of facilities to locate. Here, let's see where to locate only *one* facility using the same service radius: 

In [17]:
model = spopt.locate.MCLP.from_geodataframe(
    gdf_demand=bristol_innercity_utm,
    gdf_fac=bristol_innercity_utm,
    weights_cols='demand',
    service_radius = 4000,
    p_facilities = 1,
    demand_col="geometry", 
    facility_col="geometry",
)

In [18]:
model.solve(pulp.COIN_CMD(msg=False))

If we only locate a single facility, we get the following assignments: 

In [19]:
model.cli2fac

[[1], [1], [], [1], [1], [1], [], [1], [1]]

This means that, if we can only locate a single hospital, we would locate that hospital at site 1. And, this would leave clients 3 and 7 uncovered. To map this, we can define a variable called `covered` which is `False` when the allocation is empty:

In [20]:
covered = [assignment != [] for assignment in model.cli2fac]
covered

[True, True, False, True, True, True, False, True, True]

Now, to visualize the coverage map, we just need to visualize the `covered` variable:

In [21]:
bristol_innercity.assign(
    covered = covered
).explore('covered',
          marker_kwds=dict(radius=10),
          cmap='Dark2_r'
         )

Thus, the "covered" facilities are in the center. Facility 1 corresponds to BS2, in the center of the map (below "Kingsdown", on in of "Broadmead"). 

# Do: fit your own coverage problems!

Now it is time for you to try your hand at solving coverage problems. 

## One again, with feeling

<div class="alert alert-warning">
    
Using the methods we discussed above, can you create a new LSCP problem object to cover all Bristol post codes using a 5000 meter service radius? How many facilities are needed, and in which postcode are they located?

</div>

## Budget cuts incoming!

<div class="alert alert-warning">
    
Using the methods we discussed above, can you create a new MCLP problem object to cover as many Bristol post codes as possible with six facilities and a 5000 meter service radius? What facilities are not covered? 

</div>

## Introduce capacity

Let's assume that each facility can only serve 1000 inhabitants. Let's create a column for this in our dataframe: 

```python
bristol_postcodes['capacity'] = 1000
```

<div class="alert alert-warning">
    
Using the methods we discussed above and the `facility_capacity_col`, can you create a new LSCP problem object that includes this capacity constraint on facilities? How does it differ from the initial LSCP solution? 

</div>

## MCLP is not PMedian

<div class="alert alert-warning">

Solve a P-Median problem for the same number of facilities (six) as our previous MCLP example. How do the MCLP/P-Median model solutions differ? 

</div>

## Challenge: Bristol is far from an isometric plain! 

After a few months using the new depots, you begin to hear about complaints from your drivers about the awkwardness of the depot locations. Specifically, it seems your solver has used Euclidean Distances for everything, which means that you've assumed the Earth is flat, and trucks can drive anywhere in Bristol!  This is a problem for many of your drivers. So, you build a "cost table" that describes the __typical travel time from every postcode to every other postcode__. No doubt this can help!

The code implementing this is rather long and omitted for brevity here, but is available in `01-compute_costs.py`. We will read in the cost table here: 

In [22]:
cost_table = pandas.read_csv("./cost_table.csv")

To see the first few rows: 

In [23]:
cost_table.head()

,origin_postcode,destination_postcode,travel_cost
0,BS1,BS1,0.0
1,BS1,BS10,17665.2
2,BS1,BS11,7541.9
3,BS1,BS12,4840.7
4,BS1,BS13,2951.0


This has a typical "(origin, destination, weight)" structure as is common in storing routing data. In this data, the "travel_cost" variable measures the driving distance (in meters) between the two postcode centroids. 

We will use this to provide a bit more realistic costs for the location-allocation decisions. 

To build a cost_matrix from this, you have to use the `.pivot()` method on the `pandas.DataFrame`, like so: 

In [24]:
cost_matrix = cost_table.pivot(index="destination_postcode", columns="origin_postcode", values="travel_cost")
cost_matrix.iloc[0:5, 0:5]

origin_postcode,BS1,BS10,BS11,BS12,BS13
destination_postcode,,,,,
BS1,0.0,16534.7,7448.0,4906.5,2110.4
BS10,17665.2,0.0,15748.4,7790.1,17054.7
BS11,7541.9,11710.0,0.0,7844.6,12443.6
BS12,4840.7,7809.4,6901.1,0.0,7714.4
BS13,2951.0,17236.3,12473.6,7996.3,0.0


<div class="alert alert-warning">
    
Using `spopt.locate.LSCP.from_cost_matrix()` function, can you create a new `LSCP` problem object to cover facilities using a 1000-unit limit on the trip costs? How does this differ from your euclidean-based solution in the first question? 

</div>

## Review

Coverage problems are a kind of *spatial optimisation* problem that occur when trying to ensure that individuals can access critical infrastructure within a given travel time or distance. It requires mainly *location* decisions (where to place facilities), since *allocation* decisions (what client is served by what facility) are made implicitly based on the constraint on the maximum travel distance. As such, is it considered a fundamental *location-allocation* problem, and has structures common to many different kinds of spatial optimisation problems, such as the transportation problem or the warehouse location problem. The Location Set Cover Problem (LSCP) focuses on trying to locate as few facilities as possible in order to cover demand points within a given service distance. The Maximal Cover Location Problem (MCLP) tries to cover as many people as possible given a fixed number of facilities and service distance requirement. Coverage problem can be stated and solved easily in Python using the `spopt` library, which wraps the large amount of constraint and variable declarations required to build the $p$-Median model in the underlying `pulp` package. It supports many variants of the LSCP and MCLP problems, including cases where the client sites are not the same as the facility sites and the use of travel time/routing distances.  